# Fold Candidate Sequences with ESM3

In [1]:
import pandas as pd
import os

In [2]:
## Read in candidates data
candidates_df = pd.read_excel('../candidates.xlsx', sheet_name='Antibody Candidates')
antigens_df = pd.read_excel('../candidates.xlsx', sheet_name='Antigens')

## Get the antigen sequence for Nipah Glycoprotein G
antigen_seq = antigens_df.loc[antigens_df['antigen_id'] == 'nipah_gpG', 'antigen_sequence'].values[0]
print(f"Using Antigen Sequence: {antigen_seq}")

Using Antigen Sequence: ICLQKTSNQILKPKLISYTLPVVGQSGTCITDPLLAMDEGYFAYSHLERIGSCSRGVSKQRIIGVGEVLDRGDEVPSLFMTNVWTPPNPNTVYHCSAVYNNEFYYVLCAVSTVGDPILNSTYWSGSLMMTRLAVKPKSNGGGYNQHQLALRSIEKGRYDKVMPYGPSGIKQGDTLYFPAVGFLVRTEFKYNDSNCPITKCQYSKPENCRLSMGIRPNSHYILRSGLLKYNLSDGENPKVVFIEISDQRLSIGSPSKIYDSLGQPVFYQASFSWDTMIKFGDVLTVNPLVVNWRNNTVISRPGQSQCPRFNTCPEICWEGVYNDAFLIDRINWISAGVFLDSNQTAENPVFTVFKDNEILYRAQLASEDTNAQKTITNCFLLKNKIWCISLVEIYDTGDNVIRPKLFAVKIPEQCTH


## ...with ESM3

In [3]:
## Define sequence folding function for ESM3
from esm.sdk import client
from esm.sdk.api import ESMProtein, GenerationConfig

## Set ESM API Token in environment variable
os.environ["ESM_API_TOKEN"] = "1pIBroJm2HVLhzL5JmQ7gP"

def fold_sequence_esm3(sequence:str, model_name:str, token:str, soc:bool) -> str:
    ## Load the ESM model
    model = client(model=model_name, url="https://forge.evolutionaryscale.ai", token=token)
    ## Prepare the sequence
    sequence = sequence.replace(" ", "").replace("\n", "")
    ## Generate the structure
    try:
        input = ESMProtein(sequence=sequence, potential_sequence_of_concern=soc)
        # input = ESMProtein(sequence=sequence)
        config = GenerationConfig(track="structure", num_steps=10, temperature=0.1)
        generation = model.generate(input, config)
        protein_complex = generation.to_protein_complex()
        pdb_str = protein_complex.to_pdb_string()
        return pdb_str
    except Exception as e:
        print(f"Error folding sequence {sequence}: {e}")
        return ""

In [5]:
## For each candidate, fold the antibody-antigen complex
token = os.getenv("ESM_API_TOKEN")

for idx, row in candidates_df.iterrows():
    model = client(model="esm3-medium-multimer-2024-09", url="https://forge.evolutionaryscale.ai", token=token)
    
    antibody_id = row['antibody_id']
    print(f"Folding Candidate: {antibody_id}")

    test_chotia_pass = row['test_chotia_pass']
    if not test_chotia_pass:
        print(f"\tSkipping Candidate {row['antibody_id']} due to Chotia test failure.")
        continue

    output_pdb_path = f'../data/candidates/structures_esm3/{antibody_id}_complex.pdb'
    ## Check if the PDB already exists
    if os.path.exists(output_pdb_path):
        print(f"\tPDB for Candidate {antibody_id} already exists. Skipping folding.")
        continue

    h_chain_seq = row['h_chain']
    l_chain_seq = row['l_chain']

    sequence = f'{h_chain_seq}|{l_chain_seq}|{antigen_seq}'

    pdb_str = fold_sequence_esm3(sequence, "esm3-medium-multimer-2024-09", token, soc=True)

    with open(output_pdb_path, 'w') as f:
        f.write(pdb_str)

Folding Candidate: sbio-nipahgpg-001
	PDB for Candidate sbio-nipahgpg-001 already exists. Skipping folding.
Folding Candidate: sbio-nipahgpg-002
	PDB for Candidate sbio-nipahgpg-002 already exists. Skipping folding.
Folding Candidate: sbio-nipahgpg-003
	PDB for Candidate sbio-nipahgpg-003 already exists. Skipping folding.
Folding Candidate: sbio-nipahgpg-004
	PDB for Candidate sbio-nipahgpg-004 already exists. Skipping folding.
Folding Candidate: sbio-nipahgpg-005
	PDB for Candidate sbio-nipahgpg-005 already exists. Skipping folding.
Folding Candidate: sbio-nipahgpg-006
	Skipping Candidate sbio-nipahgpg-006 due to Chotia test failure.
Folding Candidate: sbio-nipahgpg-007
	Skipping Candidate sbio-nipahgpg-007 due to Chotia test failure.
Folding Candidate: sbio-nipahgpg-008
	PDB for Candidate sbio-nipahgpg-008 already exists. Skipping folding.


## ...with Boltz-2

In [3]:
import yaml

## Define function to create Boltz-2 YAML configuration for multimer folding

def create_boltz_yaml(h_chain_seq: str, l_chain_seq: str, antigen_seq: str, output_path: str):
    config_json = {
        "version": 1,
        "sequences": [
            {
            "protein": {
                "id": "H",
                "sequence": h_chain_seq
            }
            },
            {
            "protein": {
                "id": "L",
                "sequence": l_chain_seq
            }
            },
            {
            "protein": {
                "id": "A",
                "sequence": antigen_seq
            }
            }
        ]
        }
    
    ## Make directories if they don't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    ## Write YAML file
    with open(output_path, 'w') as f:
        yaml.dump(config_json, f)

In [5]:
## Define function for running Boltz-2 folding using the cford38 Docker image

# def run_boltz_folding(yaml_config_path: str, output_pdb_path: str):
#     docker_command = f"docker run --gpus all -v ./:/mnt/ --name boltz --rm -it cford38/boltz:2.1.1_withweights boltz predict {yaml_config_path} --out_dir /mnt/{os.path.dirname(output_pdb_path)} --use_msa_server"

#     os.system(docker_command)


## Define a function for running Boltz-2 folding from the command line
def run_boltz_folding(yaml_config_path: str, output_dir: str, accelerator: str = 'cpu'):
    boltz_command = f"boltz predict {yaml_config_path} --out_dir {os.path.dirname(output_dir)} --use_msa_server --accelerator {accelerator}"

    os.system(boltz_command)

In [ ]:
for idx, row in candidates_df.iterrows():
    antibody_id = row['antibody_id']
    print(f"Folding Candidate: {antibody_id}")
    output_dir = f'../data/candidates/structures_boltz2'

    test_chotia_pass = row['test_chotia_pass']
    if not test_chotia_pass:
        print(f"\tSkipping Candidate {row['antibody_id']} due to Chotia test failure.")
        continue

    ## Create YAML file
    yaml_config_path = f"{output_dir}/{antibody_id}.yaml"
    create_boltz_yaml(row['h_chain'], row['l_chain'], antigen_seq, yaml_config_path)

    ## Run Boltz folding
    run_boltz_folding(yaml_config_path, output_dir)

Folding Candidate: sbio-nipahgpg-001
Running on CPU, this will be slow. Consider using a GPU.
MSA server enabled: https://api.colabfold.com
MSA server authentication: no credentials provided
Checking input data.
Processing 1 inputs with 1 threads.


  0%|          | 0/1 [00:00<?, ?it/s]

Generating MSA for ../data/candidates/structures_boltz2/sbio-nipahgpg-001.yaml with 3 protein entities.
Calling MSA server for target sbio-nipahgpg-001 with 3 sequences
MSA server URL: https://api.colabfold.com
MSA pairing strategy: greedy
No authentication provided for MSA server



COMPLETE: 100%|██████████| 450/450 [elapsed: 00:00 remaining: 00:00]

100%|██████████| 1/1 [00:07<00:00,  7.54s/it]
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/colbyford/miniconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/colbyford/miniconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


Running structure prediction for 1 input.


/Users/colbyford/miniconda3/lib/python3.12/site-packages/pytorch_lightning/utilities/migration/utils.py:56: The loaded checkpoint was produced with Lightning v2.5.0.post0, which is newer than your current Lightning version: v2.5.0
/Users/colbyford/miniconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:420: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.
/Users/colbyford/miniconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Predicting DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]